### 1. Cài đặt và nhập thư viện


In [ ]:
import sys
import subprocess

def import_libraries():
    try:
        global selenium, selenium_stealth
        import selenium
        import selenium_stealth
        print("✅ Import thư viện thành công.")
        return True
    except ImportError as e:
        print(f"⚠️ Thiếu thư viện {e.name}, tiến hành cài đặt...")
        return False

# Kiểm tra và cài đặt nếu cần
if not import_libraries():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "selenium", "selenium-stealth"])
    print("✅ Cài đặt hoàn tất. Đang kiểm tra lại...")
    import_libraries()

import csv
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from queue import Queue
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium_stealth import stealth

<div style="font-size: 13px">

| 📚 Thư viện                              | 📝 Mô tả chi tiết                                                                                                                                |
| ---------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------ |
| selenium                                 | Thư viện điều khiển trình duyệt tự động, cho phép mô phỏng hành vi người dùng như click, nhập liệu, lấy dữ liệu từ trang web.                    |
| selenium_stealth                         | Dùng để "ngụy trang" trình duyệt do Selenium tạo ra, giúp tránh bị website phát hiện là bot (ví dụ như qua navigator.webdriver).                 |
| concurrent.futures.ThreadPoolExecutor    | Cho phép chạy nhiều tác vụ song song bằng luồng (thread), tăng tốc xử lý trong các tác vụ như thu thập dữ liệu từ nhiều trang web cùng lúc.      |
| threading.Lock                           | Dùng để đồng bộ giữa các luồng khi truy cập tài nguyên chung (như ghi file), tránh lỗi do nhiều luồng truy cập cùng lúc.                         |
| queue.Queue                              | Một hàng đợi an toàn khi dùng đa luồng (thread-safe), rất hữu ích khi truyền dữ liệu giữa các luồng trong quá trình thu thập hoặc xử lý dữ liệu. |
| WebDriverWait, expected_conditions (EC)  | Dùng để **chờ một điều kiện xảy ra**, ví dụ: chờ cho đến khi một phần tử xuất hiện trước khi thực hiện hành động tiếp theo.                      |
| NoSuchElementException, TimeoutException | Bắt các lỗi phổ biến khi Selenium không tìm thấy phần tử hoặc chờ quá lâu, giúp chương trình không bị crash khi gặp trang web thay đổi bất ngờ.  |

</div>


### 2. Cấu hình các tham số


In [ ]:
BASE_URL = "https://batdongsan.com.vn/ban-nha-rieng"
START_PAGE = 1
END_PAGE = 2000
OUTPUT_FILE = "nha_rieng.csv"
NUM_THREADS = 3  
MAX_RETRIES = 3 
lock = Lock()

<div style="font-size: 13px">

| 🧩 Biến / Hằng | 📝 Mô tả                                                                                                                                                                                                                  |
| -------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| NUM_THREADS    | Số luồng (threads) sẽ được sử dụng để xử lý song song. Tăng số này giúp chương trình nhanh hơn khi thu thập dữ liệu nhiều trang.                                                                                          |
| MAX_RETRIES    | Số lần thử lại tối đa nếu một thao tác (ví dụ như tải trang hoặc tìm phần tử) bị lỗi hoặc timeout.                                                                                                                        |
| lock = Lock()  | Tạo một **khóa luồng (thread lock)** để đảm bảo rằng **chỉ một luồng được thực thi đoạn mã quan trọng tại một thời điểm**, giúp tránh xung đột khi nhiều luồng cùng ghi vào một tài nguyên như file hoặc biến dùng chung. |
| BASE_URL       | URL gốc của trang web cần thu thập dữ liệu. Có thể thay đổi để thu thập các loại bất động sản khác nhau như nhà cho thuê, đất nền,...                                                                                     |
| START_PAGE     | Trang bắt đầu thu thập dữ liệu.                                                                                                                                                                                           |
| END_PAGE       | Trang kết thúc thu thập dữ liệu.                                                                                                                                                                                          |
| OUTPUT_FILE    | Tên file CSV dùng để lưu dữ liệu đã thu thập.                                                                                                                                                                             |

</div>


### 3. Cấu hình Selenium


In [ ]:
def setup_driver():
    """Khởi tạo trình duyệt với selenium-stealth để tránh bị chặn."""
    options = Options()
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--headless")  
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36")

    driver = webdriver.Chrome(options=options)

    # Dùng selenium-stealth để tránh bị phát hiện là bot
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True)

    driver.delete_all_cookies()
    return driver

<div style="font-size: 13px">

| Tham số                                          | Mô tả                                                                                                                |
| ------------------------------------------------ | -------------------------------------------------------------------------------------------------------------------- |
| 🖥️ --headless                                    | Chạy trình duyệt ẩn                                                                                                  |
| 🚫 --disable-blink-features=AutomationControlled | Giảm khả năng bị phát hiện là bot bằng cách vô hiệu hóa cơ chế kiểm tra tự động của Chrome.                          |
| 🎮 --disable-gpu                                 | Vô hiệu hóa GPU, tránh lỗi khi chạy trên máy chủ không có giao diện đồ họa.                                          |
| 🎨 --enable-unsafe-webgl                         | Kích hoạt WebGL ngay cả khi trình duyệt cho rằng nó không an toàn, giúp tránh bị phát hiện thông qua dấu vết đồ họa. |
| 🔓 --no-sandbox                                  | Loại bỏ cơ chế "sandbox" của Chrome, cần thiết khi chạy trên hệ điều hành Linux không có giao diện đồ họa.           |
| 💾 --disable-dev-shm-usage                       | Tránh lỗi hết bộ nhớ chia sẻ (/dev/shm) trên hệ thống Linux khi Chrome sử dụng quá nhiều bộ nhớ.                     |
| 🕵️‍♂️ Giả lập User-Agent                            | Giúp trình duyệt giả lập như một người dùng thật để tránh bị phát hiện là bot.                                       |
| 🎭 --disable-extensions                          | Vô hiệu hóa tất cả tiện ích mở rộng để giảm khả năng bị phát hiện là bot.                                            |
| 🧹 --disable-popup-blocking                      | Ngăn trình duyệt chặn pop-up, hữu ích khi cần tương tác với các cửa sổ bật lên.                                      |
| 🔄 --remote-debugging-port=9222                  | Cho phép gỡ lỗi từ xa, hữu ích khi cần theo dõi hành vi của trình duyệt.                                             |

</div>


### 4. Định nghĩa các hàm thu thập dữ liệu


Trong phần này, chúng ta sẽ xây dựng hai hàm chính để thu thập dữ liệu từ website bất động sản:

- `scrape_page(page, retries=0)`: Thu thập dữ liệu từ một trang cụ thể với cơ chế retry.
- `crawl_pages_concurrently(start_page, end_page, output_file)`: Chạy nhiều tác vụ thu thập song song bằng đa luồng để tăng tốc độ.


#### 4.1 Hàm scrape_page(page, retries=0)


In [ ]:
def scrape_page(page, retries=0):
    """Hàm crawl một trang với cơ chế retry."""
    driver = setup_driver()
    url = f"{BASE_URL}/p{page}"
    print(f"📄 Đang thu thập dữ liệu từ trang {page}...")
    driver.get(url)

    data = []
    try:
        wait = WebDriverWait(driver, 10, poll_frequency=0.5)
        properties = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.js__card")))

        if not properties:
            print(f"⏳ Không còn dữ liệu trên trang {page}. Dừng thu thập.")
            driver.quit()
            return None

        for prop in properties:
            try:
                title = prop.find_element(By.CSS_SELECTOR, "h3.re__card-title span").text
                price = prop.find_element(By.CSS_SELECTOR, "span.re__card-config-price").text
                area = prop.find_element(By.CSS_SELECTOR, "span.re__card-config-area").text
                
                try:
                    bedrooms = prop.find_element(By.CSS_SELECTOR, "span.re__card-config-bedroom span").text
                except NoSuchElementException:
                    bedrooms = "0"

                try:
                    toilets = prop.find_element(By.CSS_SELECTOR, "span.re__card-config-toilet span").text
                except NoSuchElementException:
                    toilets = "0"

                try:
                    location = prop.find_element(By.CSS_SELECTOR, "div.re__card-location span:last-child").text
                except NoSuchElementException:
                    location = "Không có dữ liệu"

                try:
                    published_date = prop.find_element(By.CSS_SELECTOR, "span.re__card-published-info-published-at").get_attribute("aria-label")
                except NoSuchElementException:
                    published_date = "Không có dữ liệu"

                data.append([title, price, area, bedrooms, toilets, location, published_date])

            except Exception as e:
                print(f"⚠️ Lỗi khi lấy dữ liệu từ trang {page}: {e}")

    except (TimeoutException, Exception) as e:
        if retries < MAX_RETRIES:
            print(f"⚠️ Lỗi khi crawl trang {page}, thử lại lần {retries + 1}...")
            time.sleep(random.uniform(2, 5))  
            return scrape_page(page, retries + 1) 
        else:
            print(f"❌ Đã hết số lần thử cho trang {page}. Bỏ qua trang này.")
    
    driver.quit()
    return data

- **Tham số**:

  - `page`: Số trang cần thu thập.
  - `retries`: Số lần thử lại nếu xảy ra lỗi (mặc định = 0).

- **Cơ chế xử lý**:

  - Dùng Selenium và `WebDriverWait` để chờ trang load đủ.
  - Lấy các thông tin: tiêu đề, giá, diện tích, phòng ngủ, vệ sinh, vị trí, ngày đăng.
  - Nếu không có dữ liệu hoặc bị lỗi, tự động thử lại tới `MAX_RETRIES`.

- **Kết quả trả về**:
  - Trả về danh sách dữ liệu nếu thành công.
  - Trả về `None` nếu không còn dữ liệu hoặc lỗi sau nhiều lần thử.


#### 4.2 Hàm crawl_pages_concurrently(start_page, end_page, output_file)


In [ ]:
def crawl_pages_concurrently(start_page, end_page, output_file):
    """Thu thập dữ liệu từ nhiều trang cùng lúc bằng đa luồng."""
    results = []
    queue = Queue()

    # Đưa các trang cần crawl vào hàng đợi
    for page in range(start_page, end_page + 1):
        queue.put(page)

    def worker():
        while not queue.empty():
            page = queue.get()
            data = scrape_page(page)
            if data:
                with lock:
                    with open(output_file, "a", newline="", encoding="utf-8-sig") as csvfile:
                        writer = csv.writer(csvfile)
                        writer.writerows(data)
                print(f"✅ Đã thu thập dữ liệu từ trang {page}")
            queue.task_done()

    with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
        for _ in range(NUM_THREADS):
            executor.submit(worker)

        queue.join()

    print(f"✅ Dữ liệu đã được lưu vào `{output_file}`.")

- **Tham số**:

  - `start_page`, `end_page`: khoảng trang cần crawl.
  - `output_file`: đường dẫn file CSV để lưu dữ liệu.

- **Cơ chế hoạt động**:

  - Dùng `queue.Queue` để phân phối các trang cho các luồng.
  - Mỗi luồng sẽ lấy 1 trang từ queue, gọi `scrape_page`, và ghi kết quả ra file.
  - Dùng `threading.Lock` để tránh ghi file cùng lúc gây lỗi.

- **Lợi ích**:
  - Tăng tốc độ crawl nhờ chạy song song.
  - Có thể điều chỉnh số luồng bằng `NUM_THREADS` để phù hợp tài nguyên hệ thống.


### 5. Chạy trình thu thập dữ liệu


In [ ]:
# Ghi tiêu đề CSV
with open(OUTPUT_FILE, "w", newline="", encoding="utf-8-sig") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Tiêu đề", "Giá", "Diện tích", "Số phòng ngủ", "Số nhà vệ sinh", "Địa điểm", "Ngày"])

# Bắt đầu thu thập dữ liệu
crawl_pages_concurrently(START_PAGE, END_PAGE, OUTPUT_FILE)
